 # <center> GNR652 Assignment_1   </center>
## <center> Flight Delay Prediction using Logistic Regression  </center>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import math
flight_data = pd.read_csv("FlightDelays.csv") #/content/drive/MyDrive/ML Assignment 1/
flight_data.rename(columns={'Flight Status':'Flight_Status'}, inplace=True)
print(flight_data.shape)

In [ ]:
flight_data.head(10)

In [ ]:
flight_data.info()

In [ ]:
flight_data

In [ ]:
delayed_flights = flight_data[flight_data.Flight_Status == 'delayed']

## Exploratory Data Analysis

In [ ]:
sns.countplot(x="Flight_Status", data=flight_data, hue="Flight_Status", palette = 'rocket')

In [ ]:
sns.countplot(x="DAY_WEEK", data=flight_data[flight_data.Flight_Status=='delayed'], hue="DAY_WEEK",dodge=False, palette = 'rocket')

In [ ]:
# plt.figure(figsize=(10,8))
sns.countplot(x="DEST", data=flight_data[flight_data.Flight_Status=='delayed'], hue="ORIGIN",dodge=False, palette = 'muted')

In [ ]:
# sns.pairplot(flight_data, hue="Flight_Status")

In [ ]:
sns.countplot(x="CARRIER", hue="CARRIER", data=flight_data[flight_data.Flight_Status=='delayed'],dodge=False, palette = 'Spectral')

In [ ]:
sns.countplot(data=delayed_flights,x="Weather",hue='Weather',dodge=False, palette = 'rocket')

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(x="Flight_Status", hue="DAY_WEEK", data = delayed_flights, palette='rocket')

In [ ]:
sns.boxplot(x="Flight_Status", y="CRS_DEP_TIME", data = flight_data, palette = 'rocket') 

## Getting Dummy Variables and doing Data Splitting

In [ ]:
flight_data = flight_data.drop(columns=['FL_DATE'],axis=1) #dropped as 

In [ ]:
flight_data.isnull()

In [ ]:
sns.heatmap(flight_data.isnull())

### One Hot Encoding and Label Encoding

In [ ]:
z = flight_data.drop(columns='Flight_Status')
y = pd.factorize(flight_data.Flight_Status)[0]
x = pd.get_dummies(z)
x

## Feature Selection

### By Logistic regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from sklearn.metrics import accuracy_score

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.4, random_state = 1)

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_scaled = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_scaled,y_train)

In [ ]:
y_pred = logistic_regression.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
log_odds = logistic_regression.coef_[0]
model_coef_ = pd.DataFrame(log_odds, 
             x.columns, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

In [ ]:
plt.figure(figsize=(50,20))
filter_coeff = model_coef_
bar1 = sns.barplot(data=filter_coeff, x=filter_coeff.index, y='coef', palette = 'Spectral')  
plt.xticks(rotation=90)

In [ ]:
log_odds = logistic_regression.coef_[0]
model_coef_ = pd.DataFrame(log_odds, 
             x.columns, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

In [ ]:
model_coef_3 = model_coef_[model_coef_.coef > 0.25]

In [ ]:
model_coef_3

In [ ]:
model_coef_4 = model_coef_[model_coef_.coef < -0.25]

In [ ]:
model_coef_4

In [ ]:
model_coef_5 = [model_coef_3,model_coef_4]

In [ ]:
model_coef_5 = pd.concat(model_coef_5)

###  Now we will plot the coefficients and see how are the values distributed

In [ ]:
plt.figure(figsize=(15,10))

filter_coeff = model_coef_5
bar1 = sns.barplot(data=filter_coeff, x=filter_coeff.index, y='coef', palette = 'Spectral')  
plt.xticks(rotation=90)

In [ ]:
flight_data_new = x[(model_coef_4 + model_coef_3).index]

In [ ]:
flight_data_new

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(flight_data_new,y, test_size= 0.4, random_state = 650)

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_scaled = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_scaled,y_train)

# Model Performance


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
y_pred = logistic_regression.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

In [ ]:
accuracy_score(y_test, y_pred, normalize=True)

### Cross Validation

In [ ]:
# Fit model to training data
classifier = LogisticRegression(penalty='l2', tol=1e-6, max_iter=10000)
classifier.fit(x_train, y_train)

#Cross Validation
from sklearn.model_selection import cross_val_score
print(cross_val_score(classifier,x,y,cv=10,scoring = 'accuracy').mean())
print(cross_val_score(classifier,x,y,cv=10,scoring = 'accuracy').max())

In [ ]:
cross_val_score(classifier,x,y,cv=10,scoring = 'accuracy')

## Q.6

In [ ]:
df_day6 = delayed_flights.loc[delayed_flights['DAY_WEEK'].isin(['6'])]
carriers_nodelay_6 = ['OH', 'UA', 'US', 'CO']
carriers_no_delay = df_day6.loc[df_day6['CARRIER'].isin(['OH', 'UA', 'US', 'CO'])]
origin = carriers_no_delay.loc[carriers_no_delay['ORIGIN'].isin(['DCA'])]
destination = origin.loc[origin['DEST'].isin(['LGA'])]
destination

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(data=delayed_flights, x = 'DAY_WEEK', hue = 'CARRIER', palette = 'Spectral')

In [ ]:
destination